## 1.3 YOLO(You Only Look Once)

<img src="https://miro.medium.com/max/1400/1*bSLNlG7crv-p-m4LVYYk3Q.png" width=400>

- 가장 빠른 객체 검출 알고리즘 중 하나
- Single-Stage Detection으로 기존의 detection 모델들에 비해 처리과정이 간단하기 때문에 학습과 예측의 속도가 빠름
- 모든 학습 과정이 이미지 전체를 통해 일어나기 때문에 단일 대상의 특징뿐 아니라 이미지 전체의 맥락을 학습하게 됨
- 대상의 일반적인 특징을 학습하기 때문에 다른 영역으로의 확장에서도 뛰어난 성능을 보임


### 1.1 YOLO v1

#### 특징
- 빠른 Detection 시간. 그러나 낮은 정확도 (그 뒤 SSD가 더 좋은 성능과 속도를 보였음)
- 기존의 객체 탐지는 single window나 regional proposal methods 등을 통해 바운딩 박스를 잡은 후 탐지된 바운딩 박스에 대해 분류를 수행하는 two-stage detection였음<br>
 $\rarr$ 파이프라인이 복잡하기 때문에 학습 및 예측, 최적화 느려진다는 단점 존재
- YOLO v1은 하나의 컨볼루션 네트워크를 통해 대상의 위치와 클래스를 한번에 예측 가능한 one-stage detection임
- 테두리 상자 조정(Bounding Box Coordinate)과 분류(Classification)을 동일 신경망 구조를 통해 동시에 실행하는 통합인식(Unified Detection)을 구현하는 것이 큰 특징

#### Detection 과정
1. input 이미지를 $S \times S$ grid로 분할(논문에서는 7x7)
2. grid cell 당 bounding box와 Class probability 예측  
    - 각 그리드 셀은 bouding box $B$와 해당 box의 confidence score를 예측
      - confidence score = $Pr(Object)*IoU$=객체가 존재할 확률과 IoU 값의 곱
          - 해당 box가 객체를 얼마나 포함하는지, 그 정확도는 얼마인지 등 모델이 얼마나 신뢰 가능한지를 나타내는 점수
          - box 내 객체가 존재하지 않으면 객체 존재 확률 $Pr(Object)$가 0이므로 confidence score=0
      - 각 bounding box는 $x,y,w,h,confidence \ score$로 구성($x,y$ : box 중심 좌표 / $w, h$ : box 너비, 높이)  
    - 각 그리드 셀은 조건부 클래스 확률 $Pr(Class\ i\ | \ Object)$을 예측
      - 그리드 셀이 객체를 포함할 때, 해당 객체가 i번째 클래스일 확률
      - 각 그리드 셀 당 확률값이 가장 큰 하나의 클래스만 예측
3. NMS를 통해 최종 예측
    <div style="text-align:center">
        <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FPk0eC%2FbtryCDrvbhN%2FGjM4eCM2VFV5j9nHfOKI71%2Fimg.png" width=50% height=50%>
    </div>


   - 최종 bounding box는 $S \times S \times (B*5+C)$텐서 형태로 표현
     - (S X S grid cell 크기) x ((한 grid cell당 예측할 bounding box 개수 B)*5+(class 개수 C))
     - 논문 예시로 클래스가 20개인 데이터셋에 대해 이미지를 그리드 셀 7x7 크기로 나누고, 한 그리드 셀당 box를 2개씩 예측하려고 한다면 (7x7)x(2*5+20) = 7x7x30 크기의 3차원 tensor


#### 단점
1. 작은 객체 탐지가 어려움
    - Spatial contraint (공간적 제약) : 그리드 셀 하나당 객체를 하나만 검출할 수 있어서, 한 셀에 작은 객체가 여러개 모여 있는 경우, 여러 객체를 검출하는 것이 아니라 하나로만 봐야하기 때문에 제대로 검출하지 못하는 문제
2. 새로운 aspect ratio에 대한 탐지가 어려움
    - aspect ratio : bounding box의 가로세로 비율
    - 예를 들어, 1:1, 1:2, 2:1 비율의 bounding box만 학습했는데, 3:1 비율의 box를 테스트하면 탐지 못함
3. 손실함수 Sum of Square Error(SSE)가 Bounding Box 크기와 관계없이 가중치를 동일하게 취급한 문제를 해결하지 못함
    - Bounding Box의 높이와 넓이에 루트를 취해서 크기 차이를 감소해 문제를 개선했지만, 가중치는 동일하게 주기 때문에 완전히 해결하지 못함.

#### Design and Code
<div style="text-align:center">
    <img src="./img/yolov1_number.png" width=60% height=60%>
</div>
<br>

```python
class YoLo_v1(nn.Module):
    def __init__(self, num_classes=20, num_bboxes=2):
        super(YoLo_v1, self).__init__()

        self.feature_size = 7
        self.num_bboxes=num_bboxes
        self.num_classes=num_classes
        self.conv = nn.Sequential(

            # 1
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=4),
            # nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 2
            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(192),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 3
            nn.Conv2d(in_channels=192, out_channels=128, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 4
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 5
            nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=2, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),

            # 6
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
        )

        self.fc = nn.Sequential(
            # 7
            Flatten(),
            nn.Linear(in_features=7*7*1024, out_features=4096),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Dropout(p=0.5),
            # 8
            nn.Linear(in_features=4096, out_features=(feature_size*feature_size*(5 * num_bboxes + num_classes))),
            nn.Softmax()
        )
            
        self.init_weight(self.conv)
        self.init_weight(self.fc)

    def forward(self, x):
        s, b, c = self.feature_size, self.num_bboxes, self.num_classes

        x = self.conv(x)
        x = self.fc(x)

        x = x.view(-1, s, s, (5 * b + c))
        return x

```



### 2.3.2 YOLO v2

#### 특징
1. backbone network로 `DarkNet-19`를 사용함
2. Convolutional with Anchor boxes
   - YOLO v1은 각 cell별로 2개의 bounding box를 예측하여 총 98(=7x7x2)개의 bounding box를 예측함
   - YOLO v2는 anchor box를 사용하여 보다 많은 수의 bounding box를 예측함
3. Dimension Clusters
   
   <img src="./img/dimension cluster.png" width=40% height=40%>
   
   - 더 좋은 조건으로 학습을 시작하기 위해 K-means Clustering 사용하여 최적의 anchor box 크기와 ratio를 정하도록 함
   - Ground Truth box의 width, height 값을 사용하여 K-means clustering을 수행함
   - 논문에서는 5개의 anchor box를 사용하는 것이 네트워크가 detection task를 보다 쉽게 학습할 수 있다고 함  
4. Fine-Grained Features
   
      <img src="./img/fine-Grained.png" width=40% height=40%>
   

   - YOLO v2는 최종적으로 13x13 크기의 feature map을 출력함. 이처럼 feature map의 크기가 작은 경우 큰 객체를 예측하기 용이한 반면 작은 객체는 예측하기 어렵다는 문제가 있음
   - 이 문제를 해결하기 위해 마지막 pooling을 수행하기 전에 feature map을 추출하여 26x26x512 크기의 feature map을 얻은 후 feature map의 channel은 유지하면서 4개로 분할한 후 결합하여 13x13x2048 크기의 feature map을 얻는다. <br>이러한 feature map은 보다 작은 객체에 대한 정보를 함축하고 있음
   - 이를 13x13x1024 feature map에 추가하여 13x13x3072 크기의 feature map을 얻는다.
  
5. Batch Normailzation
   - 모든 conv layer 뒤에 batch normalization을 추가함
6. High Resolution Classifier
   - YOLO v1 모델은 DarkNet을 224x224 크기로 pre-train시켰지만 detection task 시에는 448x448 크기의 이미지를 입력으로 사용함. 이는 네트워크가 object detection task를 학습하면서 동시에 새로운 입력 이미지의 resolution에 적응해야 함을 의미함
   - YOLO v2 모델은 처음부터 DarkNet을 448x448 크기로 pre-train 시켜 네트워크가 상대적으로 높은 해상도의 이미지에 적응할 시간을 제공함
7. Multi-Scale Training
   - YOLO v2 모델을 보다 강건하게 만들기 위해 다양한 입력 이미지를 사용하여 네트워크를 학습시킴
   - 논문에서는 10 batch마다 입력 이미지의 크기를 랜덤하게 선택하여 학습하도록 설계함
   - 모델은 이미지를 1/32배로 downsample시키기 때문에 입력 이미지 크기를 32배수 중에서 선택하도록 함. 
   - 320x320 크기의 이미지가 가장 작은 입력 이미지이며, 608x608 크기의 이미지가 입력될 수 있는 가장 큰 이미지임
   - 이를 통해 네트워크는 다양한 크기의 이미지를 입력받을 수 있고, 속도와 정확도 사이의 trade-off를 제공
   - 입력 이미지의 크기가 작은 경우 더 높은 FPS를 가지며, 입력 이미지의 크기가 큰 경우 더 높은 mAP 값을 가지게 됨

#### YOLO v1 VS YOLO v2
<div style="text-align:center">
  <img src="./img/diff v1,v2.png" width=40% height=40%>
</div>

#### Design and Code
<div style="text-align:center">
  <img src="./img/yolov2_number.png" width=60% height=60%>
</div>
<br>

```python
class Yolo(nn.Module):
    def __init__(self, num_classes,
                 anchors=[(1.3221, 1.73145), (3.19275, 4.00944), (5.05587, 8.09892), (9.47112, 4.84053),
                          (11.2364, 10.0071)]):
        super(Yolo, self).__init__()
        self.num_classes = num_classes
        self.anchors = anchors

        # 1
        self.stage1_conv1 = nn.Sequential(nn.Conv2d(3, 32, 3, 1, 1, bias=False), nn.BatchNorm2d(32),
                                          nn.LeakyReLU(0.1, inplace=True), nn.MaxPool2d(2, 2))
        
        # 2
        self.stage1_conv2 = nn.Sequential(nn.Conv2d(32, 64, 3, 1, 1, bias=False), nn.BatchNorm2d(64),
                                          nn.LeakyReLU(0.1, inplace=True), nn.MaxPool2d(2, 2))
        
        # 3
        self.stage1_conv3 = nn.Sequential(nn.Conv2d(64, 128, 3, 1, 1, bias=False), nn.BatchNorm2d(128),
                                          nn.LeakyReLU(0.1, inplace=True))
        self.stage1_conv4 = nn.Sequential(nn.Conv2d(128, 64, 1, 1, 0, bias=False), nn.BatchNorm2d(64),
                                          nn.LeakyReLU(0.1, inplace=True))
         
        # 4
        self.stage1_conv5 = nn.Sequential(nn.Conv2d(64, 128, 3, 1, 1, bias=False), nn.BatchNorm2d(128),
                                          nn.LeakyReLU(0.1, inplace=True), nn.MaxPool2d(2, 2))
        self.stage1_conv6 = nn.Sequential(nn.Conv2d(128, 256, 3, 1, 1, bias=False), nn.BatchNorm2d(256),
                                          nn.LeakyReLU(0.1, inplace=True))
        self.stage1_conv7 = nn.Sequential(nn.Conv2d(256, 128, 1, 1, 0, bias=False), nn.BatchNorm2d(128),
                                          nn.LeakyReLU(0.1, inplace=True))
        self.stage1_conv8 = nn.Sequential(nn.Conv2d(128, 256, 3, 1, 1, bias=False), nn.BatchNorm2d(256),
                                          nn.LeakyReLU(0.1, inplace=True), nn.MaxPool2d(2, 2))
        
        # 5                           
        self.stage1_conv9 = nn.Sequential(nn.Conv2d(256, 512, 3, 1, 1, bias=False), nn.BatchNorm2d(512),
                                          nn.LeakyReLU(0.1, inplace=True))
        self.stage1_conv10 = nn.Sequential(nn.Conv2d(512, 256, 1, 1, 0, bias=False), nn.BatchNorm2d(256),
                                           nn.LeakyReLU(0.1, inplace=True))
        self.stage1_conv11 = nn.Sequential(nn.Conv2d(256, 512, 3, 1, 1, bias=False), nn.BatchNorm2d(512),
                                           nn.LeakyReLU(0.1, inplace=True))
        
        # 6                              
        self.stage1_conv12 = nn.Sequential(nn.Conv2d(512, 256, 1, 1, 0, bias=False), nn.BatchNorm2d(256),
                                           nn.LeakyReLU(0.1, inplace=True))
        self.stage1_conv13 = nn.Sequential(nn.Conv2d(256, 512, 3, 1, 1, bias=False), nn.BatchNorm2d(512),
                                           nn.LeakyReLU(0.1, inplace=True))
        
        # 7
        self.stage2_a_maxpl = nn.MaxPool2d(2, 2)
        self.stage2_a_conv1 = nn.Sequential(nn.Conv2d(512, 1024, 3, 1, 1, bias=False),
                                            nn.BatchNorm2d(1024), nn.LeakyReLU(0.1, inplace=True))
        self.stage2_a_conv2 = nn.Sequential(nn.Conv2d(1024, 512, 1, 1, 0, bias=False), nn.BatchNorm2d(512),
                                            nn.LeakyReLU(0.1, inplace=True))
        self.stage2_a_conv3 = nn.Sequential(nn.Conv2d(512, 1024, 3, 1, 1, bias=False), nn.BatchNorm2d(1024),
                                            nn.LeakyReLU(0.1, inplace=True))
      
        # 8
        self.stage2_a_conv4 = nn.Sequential(nn.Conv2d(1024, 512, 1, 1, 0, bias=False), nn.BatchNorm2d(512),
                                            nn.LeakyReLU(0.1, inplace=True))
        self.stage2_a_conv5 = nn.Sequential(nn.Conv2d(512, 1024, 3, 1, 1, bias=False), nn.BatchNorm2d(1024),
                                            nn.LeakyReLU(0.1, inplace=True))
        self.stage2_a_conv6 = nn.Sequential(nn.Conv2d(1024, 1024, 3, 1, 1, bias=False), nn.BatchNorm2d(1024),
                                            nn.LeakyReLU(0.1, inplace=True))
        self.stage2_a_conv7 = nn.Sequential(nn.Conv2d(1024, 1024, 3, 1, 1, bias=False), nn.BatchNorm2d(1024),
                                            nn.LeakyReLU(0.1, inplace=True))
        # 11
        self.stage3_conv1 = nn.Sequential(nn.Conv2d(2048 + 1024, 1024, 3, 1, 1, bias=False), nn.BatchNorm2d(1024),
                                          nn.LeakyReLU(0.1, inplace=True))
        self.stage3_conv2 = nn.Conv2d(1024, len(self.anchors) * (5 + num_classes), 1, 1, 0, bias=False)

   def forward(self, input):
        # 1
        output = self.stage1_conv1(input)

        # 2
        output = self.stage1_conv2(output)

        # 3
        output = self.stage1_conv3(output)
        output = self.stage1_conv4(output)

        # 4
        output = self.stage1_conv5(output)
        output = self.stage1_conv6(output)
        output = self.stage1_conv7(output)
        output = self.stage1_conv8(output)

        # 5
        output = self.stage1_conv9(output)
        output = self.stage1_conv10(output)
        output = self.stage1_conv11(output)

        # 6
        output = self.stage1_conv12(output)
        output = self.stage1_conv13(output)

        residual = output # 6번 과정이 끝난 후 결과를 저장
        
        # 7
        output_1 = self.stage2_a_maxpl(output)
        output_1 = self.stage2_a_conv1(output_1)
        output_1 = self.stage2_a_conv2(output_1)
        output_1 = self.stage2_a_conv3(output_1)

        # 8
        output_1 = self.stage2_a_conv4(output_1)
        output_1 = self.stage2_a_conv5(output_1)
        output_1 = self.stage2_a_conv6(output_1)
        output_1 = self.stage2_a_conv7(output_1)
        
        # 9, 26x26x512 => 13x13x2048
        output_2 = residual
        batch_size, num_channel, height, width = output_2.data.size()
        output_2 = output_2.view(batch_size, int(num_channel / 4), height, 2, width, 2).contiguous()
        output_2 = output_2.permute(0, 3, 5, 1, 2, 4).contiguous()
        output_2 = output_2.view(batch_size, -1, int(height / 2), int(width / 2))

        # 10
        output = torch.cat((output_1, output_2), 1)
        
        # 11
        output = self.stage3_conv1(output)
        output = self.stage3_conv2(output)

        return output
```




### 2.3.3 YOLO v3

#### Design
<img src="https://postfiles.pstatic.net/MjAyMDAzMTNfMzcg/MDAxNTg0MDY3NjEyMzY4.d2fCI9WlzwaezCH3nM13-iKcLfpeyGSO9kWpPmBHzZ0g.sYbPMGPLqdGw-JDcrmNHmNzjQEFb7L-8f8IjXqXxNB4g.PNG.polpolie95/image.png?type=w966" width=40% height=40%>

<img src="https://wikidocs.net/images/page/162982/Torch_Yolo_V3_feature_map.png" width=70% height=70%>


#### 특징
- FPN와 유사한 기법을 적용하여 3개의 Feature Map Output에서(13x13, 26x26, 52x52), 각각 3개의 서로 다른 크기의 scale을 가진 anchor box로 Detection
- DarkNet-53을 통해서 좀 더 강한 Semantic한 Feature Map 추출
- Multi Labels을 예측하기 위해 Softmax가 아닌 Sigmoid 기반의 Logistic Regression Classifier로 개별 Object의 Multi labels 예측
- 수행 속도는 YOLO v2보다 줄었지만 수행 성능은 높아짐

### 2.3.4 YOLO v4

#### Design

<img src="./img/yolov4.png" width=40% height=40%>


- `Backbone` : CSPDarkNet-53 <br> 입력 이미지로부터 다양한 feature를 추출하는 역할을 함
- `Neck` : SPP(Spatial Pyramid Pooling)+PAN(Path Aggregation Network) <br> `backbone`에서 나온 결과(feature map)를 `head`로 전달하기 전에 refinement(정제), reconfiguration(재구성)함
- `Head` : YOLO v3 <br> bounding box 및 class 예측을 만드는 네트워크의 일부
- Bag of Freebies for backbone : CutMix, Mosaic, DropBlock, Class label smoothing
- Bag of Specials for backbone : Mish, CSP, Muiti-input weighted residual connections(MiWRC)
- Bag of Freebies for detector : CIoU, CmBN, DropBlock, Mosaic, Self Adversarial Training, Eliminate grid sensitivity, Using multiple anchors for a single ground truth, Cosine anneling scheduler, Optimal hyper parameters, Random training shapes
- Bag of Specials for detector : Mish, SPP, SAM, PAN, DIoU NMS

    * Bag of Freebies : 모델의 inference 시간을 늘리지 않으면서 더 높은 정확도를 얻을 수 있도록 학습시키는 방법(Data Augmentation, Semantic distribution bias, Objective function of Bounding box regression 등)
    * Bag of Special : inference 시 cost를 아주 조금 증가시키지만 정확도를 크게 향상시킬 수 있는 추가 모듈 혹은 post-processing 방법을 말함(Enhance receptive field, Attention module, Feature integration, Activation function, post-processing method 등)


<img src="https://wikidocs.net/images/page/163049/Object_Detection_Flow_Yolov4_Architecture.png" width="70%" height="70%">

<img src="https://wikidocs.net/images/page/163565/Torch_Yolo_V4_feature_map.png" width="70%" height="70%">